In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls
!pwd
%cd drive/My Drive/pytorch/

drive  sample_data
/content
/content/drive/My Drive/pytorch


# 构建简单的神经网络

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
# 获取训练设备 是否支持gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### 定义子类

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
#       # super() 函数是用于调用父类(超类)的一个方法
#       # super() 是用来解决多重继承问题的，直接用类名调用父类方法在使用单继承的时候没问题，但是如果使用多继承，会涉及
#       # 到查找顺序（MRO）、重复调用（钻石继承）等种种问题。
#       # MRO 就是类的方法解析顺序表, 其实也就是继承父类方法时的顺序表。
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
# 随机初始化一个数
X = torch.rand(1, 28, 28, device=device)
# 调用模型
logits = model(X)
# 归一化 返回【0 1】的维度
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
# argmax找出函数的最大值
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

input_image = torch.rand(3,28,28)
print(input_image.size())

tensor([[0.1031, 0.0901, 0.1054, 0.1006, 0.0941, 0.0992, 0.1006, 0.0944, 0.1081,
         0.1043]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([8])
torch.Size([3, 28, 28])


In [20]:
flatten = nn.Flatten()
print(flatten)
flat_image = flatten(input_image)
print(flat_image.size())

Flatten(start_dim=1, end_dim=-1)
torch.Size([3, 784])


In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0620,  0.4775,  0.2501, -0.0114, -0.3410,  0.7758, -0.1622, -0.2312,
         -0.1238, -0.4328,  0.1955,  0.0088, -0.1351,  0.1212,  0.5302, -0.1646,
          0.2808,  0.1176,  0.1419, -0.2882],
        [-0.0789,  0.1805,  0.1252, -0.1558, -0.6024,  0.4780, -0.0607, -0.1466,
          0.2064, -0.1578,  0.2270,  0.2820,  0.1203,  0.4900,  0.2456, -0.3436,
          0.3705,  0.3544, -0.1821, -0.2190],
        [ 0.0930,  0.2756,  0.3918, -0.1368, -0.3552,  0.2712, -0.2746,  0.1836,
          0.2031, -0.6067,  0.1908,  0.2069, -0.1442,  0.1996,  0.5487, -0.2005,
         -0.0269,  0.5263, -0.2139, -0.5162]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0620, 0.4775, 0.2501, 0.0000, 0.0000, 0.7758, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1955, 0.0088, 0.0000, 0.1212, 0.5302, 0.0000, 0.2808, 0.1176,
         0.1419, 0.0000],
        [0.0000, 0.1805, 0.1252, 0.0000, 0.0000, 0.4780, 0.0000, 0.0000, 0.2064,
         0.0000, 0.2270, 0.2820, 0.1203, 0.4900, 0.24

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0086,  0.0149,  0.0067,  ..., -0.0210, -0.0245,  0.0090],
        [-0.0013, -0.0205, -0.0269,  ...,  0.0035, -0.0159, -0.0195]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0189, -0.0243], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0158, -0.0066, -0.0052,  ..., -0.0165,  0.0114,  0.0214],
        [ 0.0237,  0.0436, -0.0027,  ...,  0.0236,  0.0177,  0.0139]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias 